# Práctica de Sistemas de Bases de Datos - Deforestación SubSahara

Partiendo del dataset facilitado por el Equipo Docente, a continuación se desarrollan las órdenes que podemos ir implementando tanto en un Terminal de línea de comandos, como en Hive, para ir resolviendo las cuestiones que se plantean en el enunciado de la Práctica.

Primero, creamos una carpeta en Hadoop.

In [105]:
! hadoop fs -mkdir deforestacionSubSahara

A continuación, incluimos en esta carpeta el dataset.

In [106]:
! hadoop fs -put /home/cloudera/deforestacionSubSahara/deforestationSubSahara.csv deforestacionSubSahara

In [107]:
! hadoop fs -ls deforestacionSubSahara

Found 1 items
-rw-r--r--   1 cloudera cloudera     546970 2019-11-22 06:34 deforestacionSubSahara/deforestationSubSahara.csv


Creamos la tabla en el Hive Editor.

En cuanto a la elección de tablas internas o externas.
Las tablas internas eliminan los datos que se utilizan de su ubicación original y los almacenan en una carpeta en el sistema de archivos de Hadoop contralada por Hive; mientras que las externas mantienen los datos en su ubicación original permitiendo que los datos estén disponibles para posibles cambios u otras herramientas de Hadoop.
Partiendo de esa base, he optado por utilizar tablas externas ya que, vamos a necesitar acceder al archivo de datos para crear tablas y vistas, y después para ejecutar la función Map que se pide en el punto 5 del enunciado de la Práctica.

In [108]:
%%writefile deforestacionSubSahara.hql

create external table deforestacionTabla (
  WorldBankRegion STRING, Country STRING, ISO3 STRING, wdpaID INT, ParkName STRING, Year INT, OutsideDeforestation DOUBLE, InsideDeforestation DOUBLE) 
ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' LOCATION '/user/cloudera/deforestacionSubSahara';

Writing deforestacionSubSahara.hql


In [109]:
! beeline -u "jdbc:hive2://localhost:10000/default" -f deforestacionSubSahara.hql

scan complete in 3ms
Connecting to jdbc:hive2://localhost:10000/default
Connected to: Apache Hive (version 1.1.0-cdh5.12.0)
Driver: Hive JDBC (version 1.1.0-cdh5.12.0)
Transaction isolation: TRANSACTION_REPEATABLE_READ
0: jdbc:hive2://localhost:10000/default> 
a (jdbc:hive2://localhost:10000/default> create external table deforestacionTabl 
LE, InsideDeforestation DOUBLE) Name STRING, Year INT, OutsideDNG, Country STRIN eforestation DOUB 
BY ',' LOCATION '/user/cloudera/deforestacionSubSahara';MITED FIELDS TERMINATED  
INFO  : Compiling command(queryId=hive_20191122063535_f7fba88d-7c2c-4cc2-800c-9170d972a980): create external table deforestacionTabla (
WorldBankRegion STRING, Country STRING, ISO3 STRING, wdpaID INT, ParkName STRING, Year INT, OutsideDeforestation DOUBLE, InsideDeforestation DOUBLE)
ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' LOCATION '/user/cloudera/deforestacionSubSahara'
INFO  : Semantic Analysis Completed
INFO  : Returning Hive schema: Schema(fieldSchemas:null, pr

Podemos comprobar si se ha creado correctamente.

In [110]:
! beeline -u "jdbc:hive2://localhost:10000/default" -e "use default; show tables;"

scan complete in 2ms
Connecting to jdbc:hive2://localhost:10000/default
Connected to: Apache Hive (version 1.1.0-cdh5.12.0)
Driver: Hive JDBC (version 1.1.0-cdh5.12.0)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=hive_20191122063535_3904f5b8-1467-4b0f-a402-8093a57ae7a8): use default
INFO  : Semantic Analysis Completed
INFO  : Returning Hive schema: Schema(fieldSchemas:null, properties:null)
INFO  : Completed compiling command(queryId=hive_20191122063535_3904f5b8-1467-4b0f-a402-8093a57ae7a8); Time taken: 0.061 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=hive_20191122063535_3904f5b8-1467-4b0f-a402-8093a57ae7a8): use default
INFO  : Starting task [Stage-0:DDL] in serial mode
INFO  : Completed executing command(queryId=hive_20191122063535_3904f5b8-1467-4b0f-a402-8093a57ae7a8); Time taken: 0.01 seconds
INFO  : OK
No rows affected (0.127 seconds)
INFO  : Compiling command(queryId=hive_2019

Ya podemos crear la vista que se pide en el punto 3 del enunciado.

In [111]:
! beeline -u "jdbc:hive2://localhost:10000/default" -e \
"create view deforestacionVista as select ISO3, ParkName, wdpaID, AVG(OutsideDeforestation) as MediaOutsideDeforestation, AVG(InsideDeforestation) as MediaInsideDeforestation from deforestacionTabla GROUP BY ISO3, ParkName, wdpaID;"

scan complete in 2ms
Connecting to jdbc:hive2://localhost:10000/default
Connected to: Apache Hive (version 1.1.0-cdh5.12.0)
Driver: Hive JDBC (version 1.1.0-cdh5.12.0)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=hive_20191122063737_b5d9a793-1a75-4b95-bd31-5159a0d297ef): create view deforestacionVista as select ISO3, ParkName, wdpaID, AVG(OutsideDeforestation) as MediaOutsideDeforestation, AVG(InsideDeforestation) as MediaInsideDeforestation from deforestacionTabla GROUP BY ISO3, ParkName, wdpaID
INFO  : Semantic Analysis Completed
INFO  : Returning Hive schema: Schema(fieldSchemas:[FieldSchema(name:iso3, type:string, comment:null), FieldSchema(name:parkname, type:string, comment:null), FieldSchema(name:wdpaid, type:int, comment:null), FieldSchema(name:mediaoutsidedeforestation, type:double, comment:null), FieldSchema(name:mediainsidedeforestation, type:double, comment:null)], properties:null)
INFO  : Completed compiling command(queryId=hive_2019

Teniendo la tabla creada, los datos ya incluidos en ella y habiendo creado la vista, ya es posible realizar las tres consultas que pide el punto 4 del enunciado.

Primera consulta. ¿Cuál es el porcentaje de deforestación interno del parque Rusizi en 2011?

In [112]:
! beeline -u "jdbc:hive2://localhost:10000/default" -e \
"select ParkName, InsideDeforestation from deforestacionTabla where ParkName = 'Rusizi' and Year = 2011;"

scan complete in 3ms
Connecting to jdbc:hive2://localhost:10000/default
Connected to: Apache Hive (version 1.1.0-cdh5.12.0)
Driver: Hive JDBC (version 1.1.0-cdh5.12.0)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=hive_20191122063838_3f5a2ff5-239f-45f4-969c-eed19c98ddd3): select ParkName, InsideDeforestation from deforestacionTabla where ParkName = 'Rusizi' and Year = 2011
INFO  : Semantic Analysis Completed
INFO  : Returning Hive schema: Schema(fieldSchemas:[FieldSchema(name:parkname, type:string, comment:null), FieldSchema(name:insidedeforestation, type:double, comment:null)], properties:null)
INFO  : Completed compiling command(queryId=hive_20191122063838_3f5a2ff5-239f-45f4-969c-eed19c98ddd3); Time taken: 0.108 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=hive_20191122063838_3f5a2ff5-239f-45f4-969c-eed19c98ddd3): select ParkName, InsideDeforestation from deforestacionTabla where Par

Segunda consulta.
¿Cuáles son los 10 parques que tienen mayor porcentaje medio de deforestación interno?

In [113]:
! beeline -u "jdbc:hive2://localhost:10000/default" -e \
"select ParkName, CAST(MediaInsideDeforestation as DECIMAL(10,2)) as MediaInsideDeforestation from deforestacionVista ORDER BY MediaInsideDeforestation DESC limit 10;"

scan complete in 2ms
Connecting to jdbc:hive2://localhost:10000/default
Connected to: Apache Hive (version 1.1.0-cdh5.12.0)
Driver: Hive JDBC (version 1.1.0-cdh5.12.0)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=hive_20191122063838_b4a33a9d-ce43-46b7-a0c5-75c39e9715e2): select ParkName, CAST(MediaInsideDeforestation as DECIMAL(10,2)) as MediaInsideDeforestation from deforestacionVista ORDER BY MediaInsideDeforestation DESC limit 10
INFO  : Semantic Analysis Completed
INFO  : Returning Hive schema: Schema(fieldSchemas:[FieldSchema(name:parkname, type:string, comment:null), FieldSchema(name:mediainsidedeforestation, type:decimal(10,2), comment:null)], properties:null)
INFO  : Completed compiling command(queryId=hive_20191122063838_b4a33a9d-ce43-46b7-a0c5-75c39e9715e2); Time taken: 0.143 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=hive_20191122063838_b4a33a9d-ce43-46b7-a0c5-75c39e9715e

Tercera consulta.
¿Cuáles son los 10 países cuyos parques tienen el mayor porcentaje de deforestación interno en 2012?

In [114]:
! beeline -u "jdbc:hive2://localhost:10000/default" -e \
"select Country, CAST(SUM(InsideDeforestation) as DECIMAL(10,2)) as MayorInsideDeforestation from deforestacionTabla where Year = 2012 GROUP BY Country ORDER BY MayorInsideDeforestation DESC limit 10;"

scan complete in 1ms
Connecting to jdbc:hive2://localhost:10000/default
Connected to: Apache Hive (version 1.1.0-cdh5.12.0)
Driver: Hive JDBC (version 1.1.0-cdh5.12.0)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=hive_20191122063939_99284eaa-5af4-41e7-9190-351238a68885): select Country, CAST(SUM(InsideDeforestation) as DECIMAL(10,2)) as MayorInsideDeforestation from deforestacionTabla where Year = 2012 GROUP BY Country ORDER BY MayorInsideDeforestation DESC limit 10
INFO  : Semantic Analysis Completed
INFO  : Returning Hive schema: Schema(fieldSchemas:[FieldSchema(name:country, type:string, comment:null), FieldSchema(name:mayorinsidedeforestation, type:decimal(10,2), comment:null)], properties:null)
INFO  : Completed compiling command(queryId=hive_20191122063939_99284eaa-5af4-41e7-9190-351238a68885); Time taken: 0.104 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=hive_20191122063939_99

Vamos a desarrollar la función map para la primera consulta, tal como pide el punto 5 del enunciado.

Implementamos y ejecutamos la función map.

In [115]:
%%writefile mapper.py
#!/usr/bin/env python
 
import sys
 
for line in sys.stdin:
	data = line.strip().split(",")
	if len(data) == 8:
		WorldBankRegion, Country, ISO3, wdpaID, ParkName, Year, OutsideDeforestation, InsideDeforestation = data
		if ParkName == "Rusizi" and Year == "2011":
			print ("%s %s" % (ParkName, InsideDeforestation))

Writing mapper.py


Para poder manejar el archivo mapper.py es necesario habilitar los permisos de acceso para lectura y escritura, y permitir que el archivo sea ejecutable.

In [117]:
! cat /home/cloudera/deforestacionSubSahara/deforestationSubSahara.csv | ./mapper.py

Rusizi 27.43


Ya por último, probamos a ejecutar el código de la función Map en Hadoop.

In [118]:
! hadoop jar /usr/lib/hadoop-mapreduce/hadoop-streaming.jar -file mapper.py -mapper mapper.py -input /user/cloudera/deforestacionSubSahara/deforestationSubSahara.csv -output /user/cloudera/deforestacionSubSahara/salidaConsulta1

19/11/22 06:41:27 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
packageJobJar: [mapper.py] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.12.0.jar] /tmp/streamjob6449263844419356830.jar tmpDir=null
19/11/22 06:41:28 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/11/22 06:41:28 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/11/22 06:41:29 INFO mapred.FileInputFormat: Total input paths to process : 1
19/11/22 06:41:29 INFO mapreduce.JobSubmitter: number of splits:2
19/11/22 06:41:29 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1574406835787_0041
19/11/22 06:41:30 INFO impl.YarnClientImpl: Submitted application application_1574406835787_0041
19/11/22 06:41:30 INFO mapreduce.Job: The url to track the job: http://quickstart.cloudera:8088/proxy/application_1574406835787_0041/
19/11/22 06:41:30 INFO mapreduce.Job: Running job: job_1574406835787_0041
19/11/22 06:41:37 INFO m

In [119]:
! hadoop fs -ls deforestacionSubSahara

Found 2 items
-rw-r--r--   1 cloudera cloudera     546970 2019-11-22 06:34 deforestacionSubSahara/deforestationSubSahara.csv
drwxr-xr-x   - cloudera cloudera          0 2019-11-22 06:41 deforestacionSubSahara/salidaConsulta1


In [120]:
! hadoop fs -ls deforestacionSubSahara/salidaConsulta1

Found 2 items
-rw-r--r--   1 cloudera cloudera          0 2019-11-22 06:41 deforestacionSubSahara/salidaConsulta1/_SUCCESS
-rw-r--r--   1 cloudera cloudera         14 2019-11-22 06:41 deforestacionSubSahara/salidaConsulta1/part-00000


In [121]:
! hadoop fs -cat deforestacionSubSahara/salidaConsulta1/part-00000

Rusizi 27.43	
